# Albero di decisione
L'idea è quella di progettare un classificatore basato su una serie di test sulle caratteristiche in cui i test vengono scelti in funzione delle risposte precedenti. Si ottiene una struttura ad albero in cui i nodi interni sono relativi ai test, gli archi corrispondono agli esiti del test, e alle foglie corrispondono le decisioni (le classi).

!["Albero"](albero_decisione_iris.png)

La figura mostra un classificatore del dataset Iris: in ogni nodo è riportato il test nella forma f_i < val dove i indica la colonna relativa alla caratteristica coinvolta.

Supponiamo che le decisioni siano binarie e che venga testata una caratteristica alla volta. Il modello si addestra in modo che ad ogni nodo corrisponda l'insieme di esempi per i quali tutti i test nei nodi dalla radice sono soddisfatti. Il nuovo test è eseguito sulla feature con il maggior "guadagno informativo" rispetto agli esempi relativi al nodo.

Ma cosa si intende per **guadagno informativo**?
Sia $D_p$ l'insieme di esempi relativi al nodo $p$ e $n$ la sua cardinalità. Inoltre sia $n_c$ il numero di esempi in $D_p$ appartenenti alla classe $c$. Supponiamo di classificare un esempio $x$ in $D_p$ usando come distribuzione di probabilità quella implicata dalla distribuzione delle classi in $D_p$ allora la probabilità di classificare male $x$ è data:
$$I_G(D_p)=\sum_c \frac{n_c}{n}(1-\frac{n_c}{n})$$

in quanto $\frac{n_c}{n}$ è la probabilità di assegnare la classe $c$ e che $x$ siappartenga alla classe $c$ mentre $1-\frac{n_c}{n}$ è la probabilità che $x$ non appartenga a quella classe. Il suo valore è basso se una classe è più numerosa delle altre. Il valore è massimo se le classi sono equidistribuite.

Sia $D_L$ e $D_R$ la partizione di $D_p$ nei suoi nodi figli: Se $I_G(D_L)$ e $I_G(D_R)$ sono simili a $I_G(D_p)$, la probabilità di sbagliare classificazione resta la stessa anche dopo il test sul nodo $p$, ovvero il test su $p$ non ha portato grossi vantaggi. Un vantaggio lo otteniamo se quei due valori sono piccoli ovvero se:
$$IG(DP)=I_G(D_p)-\big(\frac{|D_L|}{n}I_G(D_L)+\frac{n-|DL|}{n}I_G(D_R)\big)$$

è massima. $IG$ è detta **guadagno informativo**.

Concludendo, in fase di addestramento (creazione dell'albero), si sceglierà la feature e il valore, che implica una partizione di $D_p$che renda massimo il guadagno informativo.

Oltre alla funzione di Gini, un altra misura di intercetezza è l'**entropia**:

$$I_H(D_p)=-\sum_c \frac{n_c}{n}\log_2(\frac{n_c}{n})$$

Durate l'addestramento, un nodo diventa foglia se gli esempi ad esso associati appartengono alla stessa classe $c$ : questa sarà la classe con cui verrà classificato $x$ se la catena di condizioni su $x$ terminano su questa foglia.

Si osservi che l'insieme di vincoli dell'albero determina delle regioni di decisioni rettangolari che possono diventare sempre più piccole man mano che la profondità dell'albero aumenta. In casi estremi si potrebbe giungere ad una regione rettangolare per ogni esempio: questo classificherebbe in modo corretto tutti gli esempi utilizzati per il training ma generalizzerebbe male (overfitting). Per questo motivo la profondità dell'albero è limitata a valori tra 3 e 4. Quando un nodo raggiunge la massima profondità consentita diventa foglia e l'etichetta gli verrà assegnata a maggioranza tra quelle del dataset corrispondente al nodo. Un nodo diventa foglia anche quando la dimensione del dataset è al di sotto di una certa soglia, anche i questo caso l'etichetta viene assegnata a maggioranza.

La funzione di impurità, l'altezza massima dell'albero, e la dimensione minima del dataset costituiscono gli iperparametri del modello.

## Implementazione

Un *nodo interno* è un dizionario con i seguenti campi:

+ *'index'*: l'indice della colonna di $X$ corrispondente alla caratteristica usata per il test;
+ *'value'*: il valore con cui confrontare la caratteristica;
+ *'groups'*: una coppia che contiene  $D_L$ e $D_R$;
+ 'left, 'right': i riferimenti ai due figli;

Un *nodo foglia* è semplicemente un valore, il nome della classe.

## Codice

In [1]:
import os
import pandas as pd
import numpy as np
from graphviz import Digraph

In [2]:
class DecisionTree(object):
    def __init__(self, max_depth=3, min_size=1):
        self.max_depth = max_depth
        self.min_size = min_size
        self.tree = None
        self._impurity_fun = self._gini

    def fit(self, X, y):
        """ Costruisce l'albero di decisione """
        y = np.array(y).reshape(-1, 1)  # una colonna x tutte le righe che servono
        
        '''
        dataset contiene sia X che y impilati verticalmente, questa è 
        la soluzione più conveniente per semplificare le operazioni
        di filtro delle righe che porterà alle suddivisioni del dataset che
        definiranno i nodi dell'albero
        '''
        dataset =  np.hstack((X, y)) # Concatenazione orizzontale
        self.tree = self._build_tree(dataset, 1)

    def _info_gain(self, dataset, groups):
        nl, nr = groups[0].shape[0], groups[1].shape[0]
        n = nl + nr
        ig = self._impurity_fun(dataset) - self._impurity_fun(groups[0])*nl/n - self._impurity_fun(groups[1])*nr/n
        return ig
        
    def _entropy(self, dataset):
        labs, occur = np.unique(dataset[:,-1], return_counts=True)
        score = 0
        for i, _ in enumerate(labs):
            proportion = occur[i] / dataset.shape[0]
            score += proportion * np.log2(proportion)
        return -score
    
    def _gini(self, dataset):
        labs, occur = np.unique(dataset[:,-1], return_counts=True)
        score = 0
        for i, _ in enumerate(labs):
            proportion = occur[i] / dataset.shape[0]
            score += proportion ** 2
        return 1-score


    def _split_dataset(self, index, value, dataset):
        """ Divide il dataset in due gruppi in base al confronto della caratteristica
        index con value"""
        mask = dataset[:, index] < value
        left, right = dataset[mask], dataset[~mask]

        return left, right

    def _get_best_split(self, dataset):
        """ Trova la feature (colonna di dataset) sulla quale esiste un valore tale che  
        Massimizza il guadagno informativo su tutte le possibili suddivisioni ottenibili usando
        tutte le possibili caratteristiche.

        Quindi per ogni caratteristica index e per ogni esempio row, si divide il dataset
        in base al test x[index] < row[index] e se ne calcola  il guadagno informativo.
        Si sceglie indx e row[index] in modo da massimizzare questo valore 
        """
        best_index, best_value, best_score, best_groups = None, None, float('-inf'), None
        for index in range(dataset.shape[1] - 1):  # Escludiamo la colonna target
            for row in dataset:
                groups = self._split_dataset(index, row[index], dataset)
                ig = self._info_gain(dataset, groups)
                if ig > best_score:
                    best_index, best_value, best_score, best_groups = index, row[index], ig, groups

        # ritorna un nodo
        return {'index': best_index, 'value': best_value, 'groups': best_groups}

    def _create_leaf(self, group):
        """ Crea un nodo foglia con la classe più comune """
        values, counts = np.unique(group[:,-1], return_counts=True)
        return values[np.argmax(counts)]

    def _split(self, node, depth):
        """ Cresce l'albero ricorsivamente """
        left, right = node['groups']
        #del node['groups']
        
        # Se uno dei gruppi ï¿½ vuoto, assegniamo una foglia
        if left.size == 0 or right.size == 0:
            node['left'] = node['right'] = self._create_leaf(np.vstack( (left, right) ))
            return

        # Fermiamo la crescita se abbiamo raggiunto la profonditï¿½ massima
        if depth >= self.max_depth:
            node['left'], node['right'] = self._create_leaf(left), self._create_leaf(right)
            return

        # Se il gruppo sinistro ï¿½ troppo piccolo, creiamo una foglia
        if len(left) <= self.min_size:
            node['left'] = self._create_leaf(left)
        else:
            node['left'] = self._get_best_split(left)
            self._split(node['left'], depth + 1)

        # Se il gruppo destro ï¿½ troppo piccolo, creiamo una foglia
        if len(right) <= self.min_size:
            node['right'] = self._create_leaf(right)
        else:
            node['right'] = self._get_best_split(right)
            self._split(node['right'], depth + 1)

    def _build_tree(self, dataset, depth):
        """ Costruisce l'albero a partire dai dati """
        root = self._get_best_split(dataset)
        self._split(root, depth)
        return root

    def _is_leaf(self, node):
        return not isinstance(node, dict)

    def _predict_example(self, node, row):
        """ Predice il valore di una singola riga """
        if row[node['index']] < node['value']:
            if self._is_leaf(node['left']):
                return node['left']
            else:
                return self._predict_example(node['left'], row)
                
        else:
            if self._is_leaf(node['right']):
                return node['right']
            else:
                return self._predict_example(node['right'], row)
            
                
            
    def predict(self, row):
        """ Predice la classe di una singola riga """
        return self._predict_example(self.tree, row)

    def predict_batch(self, X):
        """ Predice su un intero dataset """
        return [self.predict(row) for row in X]
    
    def draw_tree(self):
        self.the_tree = Digraph()
    
        def add_nodes_edges(node, parent_id=None, edge_lab = 'SI'):
            if node is None:
                return
    
            # Se foglia (intero)
            if self._is_leaf(node):
                node_id = str(id(node))
                self.the_tree.node(node_id, str(node))
                if parent_id:
                    self.the_tree.edge(parent_id, node_id, edge_lab)
                return
    
            # Nodo interno
            node_id = str(id(node))
            label = f"f_{str(node.get('index',''))} < {str(node.get('value', ''))}" 
            self.the_tree.node(node_id, label)
    
            if parent_id:
                self.the_tree.edge(parent_id, node_id, edge_lab)
    
            add_nodes_edges(node.get('left'), node_id, 'SI')
            add_nodes_edges(node.get('right'), node_id, 'NO')
    
        add_nodes_edges(self.tree)

    def show_tree(self):
        return self.the_tree

In [3]:
s = os.path.join('dataset', 'iris.data')
df = pd.read_csv(s,
                 header=None,
                 encoding='utf-8')

### Esempi

In [4]:
y = df.iloc[:100, 4].values
y = np.where(y == 'Iris-versicolor', 1, -1)

# extract sepal length and petal length
X = df.iloc[:100, [0, 2]].values

X_std = (X-X.mean(0))/X.std(0)

tree = DecisionTree(max_depth=3, min_size=1)
tree.fit(X,y)
np.mean([tree.predict_batch(X) == y])

np.float64(1.0)

In [5]:
# [0:50] iris-setosa
# [50:100] iris-versicolor
# [100:150] iris-virginica
y = df.iloc[50:, 4].values
y = np.where(y == 'Iris-versicolor', 1, -1)

# extract sepal length and petal length
X = df.iloc[50:, [0, 2]].values

X_std = (X-X.mean(0))/X.std(0)

tree = DecisionTree(max_depth=3, min_size=1)
tree.fit(X,y)
np.mean([tree.predict_batch(X) == y])

np.float64(0.97)

In [6]:
tree.draw_tree()
tree.show_tree()

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [7]:
y = df.iloc[:, 4].values

X = df.iloc[:, [0, 1, 2, 3]].values

X_std = (X-X.mean(0))/X.std(0)

tree = DecisionTree(max_depth=3, min_size=1)
tree.fit(X,y)
np.mean([tree.predict_batch(X) == y])

np.float64(0.9733333333333334)

In [8]:
tree.draw_tree()
t = tree.show_tree()
t


ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH